In [1]:
import os
from PIL import Image
import pandas as pd

directorios=[r"maize\NOR", r"maize/AP", r"maize/BN", r"maize/FM", r"maize/HD", r"maize/MY", r"maize/SD"]
df=pd.DataFrame()
for element in directorios:
    # Directorio donde se encuentran las imágenes
    directorio_imagenes = r"D:/Data Science/grainspace"+r"/"+element

    # Lista para almacenar las dimensiones de las imágenes
    dimensiones = []

    no_image=0
    image_count = 0
    # Recorre todos los archivos en el directorio de imágenes
    for archivo in os.listdir(directorio_imagenes):
        # Asegúrate de que el archivo es una imagen
        if archivo.endswith('.jpg') or archivo.endswith('.png'):
            # Abre la imagen
            imagen = Image.open(os.path.join(directorio_imagenes, archivo))
            # Obtiene las dimensiones de la imagen
            ancho, alto = imagen.size
            # Añade las dimensiones a la lista
            dimensiones.append({'nombre': archivo, 'ancho': ancho, 'alto': alto})
            image_count+=1

        else: no_image+=1
        

    # Crea un DataFrame de pandas a partir de la lista de dimensiones
    df_temp =pd.DataFrame(dimensiones)
    df=pd.concat([df,df_temp])
    print(image_count,no_image)
# Guarda el DataFrame como un archivo CSV
df.to_csv('dimensiones_imagenes.csv', index=False)


400 0
100 0
200 0
100 0
120 0
300 0
40 0


In [2]:
print(df.head(5))


                                              nombre  ancho  alto
0  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    262   400
1  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    273   333
2  20-UW-M600-0010_5_20211020103353_95_00_06_UD_1...    240   346
3  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    252   338
4  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    304   442


In [3]:
print(df.describe(),"\n")
print("Los valores maximos son:\n",df.max(),"\n Los minimos son:\n", df.min())

             ancho         alto
count  1260.000000  1260.000000
mean    258.492857   316.719841
std      42.489497    52.443700
min     101.000000   161.000000
25%     237.000000   286.000000
50%     263.000000   325.000000
75%     287.000000   354.000000
max     374.000000   450.000000 

Los valores maximos son:
 nombre    20-UW-M600-0010_5_20211020153154_9_00_06_UD_27...
ancho                                                   374
alto                                                    450
dtype: object 
 Los minimos son:
 nombre    20-UW-M600-0010_5_20211019105231_35_00_06_UD_1...
ancho                                                   101
alto                                                    161
dtype: object


las imagenes analizadas de diferentes tamaños seran usadas en un modelo de ML, se debe cambiar el tamaño de cada una y normalizar.
Para ello se va a usar la biblioteca Pillow para el cambio de tamaño y OpenCV para la normalización.

In [4]:
import os
from PIL import Image
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm


df2=pd.DataFrame()
for element in tqdm(directorios):
    # Directorio donde se encuentran las imágenes
    directorio_imagenes = r"D:/Data Science/grainspace"+r"/"+element

    # Tamaño deseado para las imágenes
    tamaño_deseado = (101, 161)

    # Lista para almacenar las dimensiones de las imágenes
    dimensiones = []

    # Recorre todos los archivos en el directorio de imágenes
    for archivo in tqdm(os.listdir(directorio_imagenes)):
        # Asegúrate de que el archivo es una imagen
        if archivo.endswith('.jpg') or archivo.endswith('.png'):
            # Abre la imagen
            imagen = Image.open(os.path.join(directorio_imagenes, archivo))
            # Cambia el tamaño de la imagen
            imagen = imagen.resize(tamaño_deseado)
            # Convierte la imagen a una matriz numpy
            imagen_np = np.array(imagen)
            # Normaliza la imagen al rango 0-255
            imagen_normalizada = cv2.normalize(imagen_np, None, 0, 255, cv2.NORM_MINMAX)
            # Añade las dimensiones a la lista
            dimensiones.append({'nombre': archivo, 'ancho': imagen_normalizada.shape[1], 'alto': imagen_normalizada.shape[0]})
            
            # Directorio donde se guardarán las imágenes modificadas
            directorio_salida = r"D:/Data Science/grainspace/pre"+r"/"+element
            # Asegúrate de que el directorio de salida existe
            try:
                os.makedirs(directorio_salida, exist_ok=True)
            except:
                os.makedirs(directorio_salida)
            # Guarda la imagen normalizada
            imagen_salida = Image.fromarray(imagen_normalizada)
            imagen_salida.save(os.path.join(directorio_salida, archivo))

            

     # Crea un DataFrame de pandas a partir de la lista de dimensiones
    df_temp =pd.DataFrame(dimensiones)
    df2=pd.concat([df2,df_temp])
    # Guarda el DataFrame como un archivo CSV
    df2.to_csv('dimensiones_Finales.csv', index=False)


    


100%|██████████| 7/7 [00:12<00:00,  1.78s/it]


In [5]:
print(df2.head(5))
print(df2.describe(),"\n")
print("Los valores maximos son:\n",df2.max(),"\n Los minimos son:\n", df2.min())


                                              nombre  ancho  alto
0  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    101   161
1  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    101   161
2  20-UW-M600-0010_5_20211020103353_95_00_06_UD_1...    101   161
3  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    101   161
4  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    101   161
        ancho    alto
count  1260.0  1260.0
mean    101.0   161.0
std       0.0     0.0
min     101.0   161.0
25%     101.0   161.0
50%     101.0   161.0
75%     101.0   161.0
max     101.0   161.0 

Los valores maximos son:
 nombre    20-UW-M600-0010_5_20211020153154_9_00_06_UD_27...
ancho                                                   101
alto                                                    161
dtype: object 
 Los minimos son:
 nombre    20-UW-M600-0010_5_20211019105231_35_00_06_UD_1...
ancho                                                   101
alto                                        

Ahora nos aseguramos del exito, repitiendo el primer script a las nuevas imagenes.

In [6]:
df3=pd.DataFrame()
for element in tqdm(directorios):
    # Directorio donde se encuentran las imágenes
    directorio_imagenes2 = r"D:/Data Science/grainspace/pre"+r"/"+element

    # Lista para almacenar las dimensiones de las imágenes
    dimensiones = []

    no_image=0
    image_count = 0
    # Recorre todos los archivos en el directorio de imágenes
    for archivo in os.listdir(directorio_imagenes2):
        # Asegúrate de que el archivo es una imagen
        if archivo.endswith('.jpg') or archivo.endswith('.png'):
            # Abre la imagen
            imagen = Image.open(os.path.join(directorio_imagenes2, archivo))
            # Obtiene las dimensiones de la imagen
            ancho, alto = imagen.size
            # Añade las dimensiones a la lista
            dimensiones.append({'nombre': archivo, 'ancho': ancho, 'alto': alto})
            image_count+=1

        else: no_image+=1
        

    # Crea un DataFrame de pandas a partir de la lista de dimensiones
    df_temp =pd.DataFrame(dimensiones)
    df3=pd.concat([df3,df_temp])
    print(image_count,no_image)
# Guarda el DataFrame como un archivo CSV
df3.to_csv('dimensiones_imagenes.csv', index=False)

 14%|█▍        | 1/7 [00:01<00:07,  1.20s/it]

400 0


 29%|██▊       | 2/7 [00:01<00:03,  1.50it/s]

100 0


 43%|████▎     | 3/7 [00:02<00:02,  1.59it/s]

...

100 0


 71%|███████▏  | 5/7 [00:02<00:00,  2.25it/s]

120 0


100%|██████████| 7/7 [00:03<00:00,  1.88it/s]

300 0
40 0


In [10]:
print(df3.head(5))
print(df3.describe(),"\n")
print("Los valores maximos son:\n",df3.max(),"\n Los minimos son:\n", df3.min())

                                              nombre  ancho  alto
0  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    101   161
1  20-UW-M600-0010_5_20211020103317_101_00_06_UD_...    101   161
2  20-UW-M600-0010_5_20211020103353_95_00_06_UD_1...    101   161
3  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    101   161
4  20-UW-M600-0010_5_20211020103353_95_00_06_UD_2...    101   161
        ancho    alto
count  1260.0  1260.0
mean    101.0   161.0
std       0.0     0.0
min     101.0   161.0
25%     101.0   161.0
50%     101.0   161.0
75%     101.0   161.0
max     101.0   161.0 

Los valores maximos son:
 nombre    20-UW-M600-0010_5_20211020153154_9_00_06_UD_27...
ancho                                                   101
alto                                                    161
dtype: object 
 Los minimos son:
 nombre    20-UW-M600-0010_5_20211019105231_35_00_06_UD_1...
ancho                                                   101
alto                                        